In [4]:
from rich import print
import asyncpg
from dotenv import load_dotenv
load_dotenv()
from os import environ
DB_CONN = None
async def create_db_conn() -> asyncpg.Connection:
    global DB_CONN
    if not DB_CONN:
        DB_CONN = await asyncpg.connect(
            environ['DATABASE_URL']
        )
    return DB_CONN

In [5]:
await create_db_conn()

In [ ]:
cursor: asyncpg.Connection = DB_CONN

In [27]:
await cursor.fetch("""SELECT "_ListToStory"."A", COUNT(*) FROM "_ListToStory" LEFT JOIN "Story" ON "_ListToStory"."B" = "Story".id WHERE "Story".completed = true GROUP BY "A";""")

[<Record A=1459396880 count=3>,
 <Record A=32842746 count=60>,
 <Record A=1527428009 count=24>,
 <Record A=957928338 count=4>,
 <Record A=1253367240 count=47>,
 <Record A=756745336 count=7>,
 <Record A=429439257 count=11>,
 <Record A=952994522 count=3>,
 <Record A=504970183 count=9>,
 <Record A=1479706805 count=2>,
 <Record A=1033679619 count=6>,
 <Record A=422089749 count=10>,
 <Record A=1350062097 count=6>,
 <Record A=1101926375 count=5>,
 <Record A=806978172 count=63>,
 <Record A=1209741655 count=5>,
 <Record A=1444957233 count=143>,
 <Record A=944614134 count=1>,
 <Record A=672894736 count=14>,
 <Record A=400760165 count=62>,
 <Record A=1599263331 count=37>,
 <Record A=924758999 count=138>,
 <Record A=1086866138 count=37>,
 <Record A=1269215588 count=10>,
 <Record A=1178987768 count=35>,
 <Record A=916207304 count=2>,
 <Record A=587819429 count=1>,
 <Record A=497238588 count=32>,
 <Record A=1022239178 count=7>,
 <Record A=1114384337 count=7>,
 <Record A=629131105 count=30>,
 <Recor

In [32]:
await cursor.fetch("""SELECT * FROM "_ListToStory" INNER JOIN "Story" ON "_ListToStory"."B"="Story".id WHERE "Story".completed=true LIMIT 1;""")


[<Record A=758237883 B=60356463 id=60356463 title='instagram ( tyler hoechlin . )' description='In a world where Tyler Hoechlin actually uses instagram\n  \n   ( started; january 18th 2016\n  ended; may 29th 2016 )' length=124927 createDate=datetime.datetime(2016, 1, 18, 16, 20, 14) modifyDate=datetime.datetime(2020, 9, 25, 18, 34, 4) voteCount=5832 readCount=198820 commentCount=1153 language='{"id": 1, "name": "English"}' username='tposeys' cover='https://img.wattpad.com/cover/60356463-256-k785884.jpg' cover_timestamp=datetime.datetime(2017, 7, 23, 11, 58, 4) completed=True categories=[6, 0] tags=['ardencho', 'danielsharman', 'dylanobrien', 'dylansprayberry', 'hollandroden', 'ianbohen', 'instagram', 'katgraham', 'khylinrhambo', 'otp', 'shelleyhennig', 'socialmedia', 'teenwolf', 'texttospeech', 'tylerhoechlin', 'tylerposey'] rating=1 mature=False copyright=1 url='https://www.wattpad.com/story/60356463-instagram-tyler-hoechlin' firstPublishedPart='{"id": 209664668, "createDate": "2016-0

In [ ]:
records = await cursor.fetch("""SELECT "_ListToStory"."A", ARRAY_AGG("_ListToStory"."B") AS "StoryIDs", COUNT("_ListToStory"."A") FROM "_ListToStory" INNER JOIN "Story" ON "_ListToStory"."B"="Story".id WHERE "Story".completed=true AND "Story"."voteCount" > 100 GROUP BY "_ListToStory"."A" HAVING COUNT("_ListToStory"."A") > 10 LIMIT 1;""")
print(records[:10])
print(len(records))

[
    <Record A=1448628 StoryIDs=[6501, 17445, 20412, 36197, 46740, 51852, 52438, 61452, 64013, 65368, 66901, 67877, 
70993, 76149, 77791, 78825, 80076, 96020, 130587, 149887, 175458, 212492, 316559, 334332, 334542, 469624, 475879, 
674163, 715089, 750260, 806525, 826740, 827785, 877899, 974868, 1001445, 1188656, 1383487, 1480279, 1585879, 
1994914, 2392257, 2426517, 3773167, 4388430, 34879301] count=46>
]

1

In [ ]:
# Gather lists that have 10 or more stories, and each of those stories have atleast 10 votes (completed or otherwise). Return the list ID and a list of all stories in that list.

if True:
    records = await cursor.fetch("""
    SELECT
        "_ListToStory"."A" as "list",
        ARRAY_AGG("_ListToStory"."B") AS "stories",
        COUNT("_ListToStory"."A") AS "num_stories"
        FROM "_ListToStory"

        INNER JOIN "Story"
        ON "_ListToStory"."B" = "Story".id
        GROUP BY "_ListToStory"."A"
        HAVING MIN("Story"."voteCount") >= 10
        AND COUNT("_ListToStory"."A") >= 10

        ORDER BY "_ListToStory"."A" ASC
        LIMIT 50
        OFFSET 50;

    """)
    print(records[:10])
    print(len(records))

[
    <Record count=20 list=1576380 stories=[29741, 45333, 87733, 150342, 164023, 274358, 290325, 298630, 312308, 
422784, 453813, 461232, 577596, 592628, 617189, 666835, 948340, 973477, 1081752, 1994914] num_stories=20>,
    <Record count=13 list=1580298 stories=[55637, 77870, 77925, 79790, 172590, 317391, 380579, 414323, 658747, 
844274, 1646513, 1646598, 8388971] num_stories=13>,
    <Record count=19 list=1590726 stories=[3955081, 4250659, 9880907, 10248226, 10648165, 11858675, 11972685, 
13864631, 25446980, 32285874, 37336268, 53548111, 69196404, 91773253, 98764116, 107087772, 115360648, 137249969, 
147628380] num_stories=19>,
    <Record count=12 list=1601185 stories=[501071, 680878, 1001445, 1081752, 1609592, 3269344, 6902011, 7877337, 
7943640, 10635790, 14546507, 21220077] num_stories=12>,
    <Record count=27 list=1601483 stories=[49609, 418406, 861947, 1369850, 2549896, 4124618, 7467918, 11246794, 
11759278, 13288378, 16094526, 22574920, 31481213, 32706159, 34628127, 45466050, 49989757, 52489700, 63724577, 
70301911, 83484297, 96088215, 102121568, 108352597, 112942839, 122621999, 133344241] num_stories=27>,
    <Record count=23 list=1604540 stories=[186, 61255, 286347, 420503, 547597, 577593, 581543, 701726, 775456, 
880283, 995344, 1049010, 1206042, 1227626, 1561375, 1980657, 2541200, 5399509, 19140246, 62808185, 192620310, 
220998019, 221906141] num_stories=23>,
    <Record count=51 list=1605494 stories=[33934, 60676, 105872, 128046, 143799, 176238, 203382, 241271, 248131, 
430006, 465702, 588570, 1029016, 1303405, 1394311, 1869329, 5948613, 8362022, 8681877, 10977308, 12710832, 
22789992, 25097001, 29969320, 33601669, 36308499, 38820105, 40401270, 42741484, 42956020, 43459826, 46159699, 
52428665, 54221457, 54708616, 55220571, 56537585, 60146790, 60272741, 63445259, 64927784, 65523524, 73447584, 
82489830, 85002756, 85423314, 88620515, 92003977, 106922825, 122929383, 138321765] num_stories=51>,
    <Record count=17 list=1610084 stories=[15427, 41030, 80076, 80990, 84307, 143799, 295466, 325608, 337978, 
390868, 391172, 465702, 567980, 786028, 826740, 1035573, 6532949] num_stories=17>,
    <Record count=21 list=1610984 stories=[325608, 340975, 391172, 557655, 652282, 3333821, 14862919, 16776720, 
16876944, 17836643, 21489080, 22644562, 24904182, 28348726, 44730719, 49206761, 82262148, 88620515, 98282293, 
233247755, 299198752] num_stories=21>,
    <Record count=21 list=1612810 stories=[73997, 87476, 172590, 233820, 240356, 247896, 293792, 317391, 346066, 
346249, 394481, 408916, 414323, 455340, 465702, 503223, 506150, 526513, 550572, 611184, 1239878] num_stories=21>
]

50

In [117]:
# Gather lists that have 10 or more stories, and each of those stories have atleast 10 votes (completed or otherwise). Return the list ID and a list of all stories in that list.

limit, offset = 5000, 0
with open('sids.txt', 'a') as sids_h:
    with open('lids.txt', 'a') as lids_h:
        while True:
            records = await cursor.fetch("""
            SELECT
                "_ListToStory"."A" as "list",
                ARRAY_AGG("_ListToStory"."B") AS "stories"
                FROM "_ListToStory"

                INNER JOIN "Story"
                ON "_ListToStory"."B" = "Story".id
                GROUP BY "_ListToStory"."A"
                HAVING MIN("Story"."voteCount") >= 10
                AND COUNT("_ListToStory"."A") >= 10

                ORDER BY "_ListToStory"."A" ASC
                LIMIT $1
                OFFSET $2;

            """, limit, offset)
            lids, sids = [str(record['list']) for record in records], [record['stories'] for record in records]
            for sid in sids:
                sids_h.write(" ".join([str(s) for s in sid]) + "\n")
            lids_h.write("\n".join(lids) + "\n")
            print(f"done {limit=} {offset=}")
            offset = offset + limit
            if not records:
                break
print("done")



done limit=5000 offset=0

done limit=5000 offset=5000

done limit=5000 offset=10000

done limit=5000 offset=15000

done limit=5000 offset=20000

done limit=5000 offset=25000

done limit=5000 offset=30000

done

In [124]:
from itertools import chain


with open('sids.txt', 'r') as story_ids:
    ids = list(chain.from_iterable([line.split(" ") for line in story_ids.readlines()]))
    out = "\n".join([id for id in ids if id != '']).replace("\n\n", "\n") # no idea why this happens
    with open('story_ids.txt', 'w') as handler:
        handler.write(out)

In [132]:
with open('embeddings') as handler:
    embeddings_raw = [line.strip() for line in handler.readlines()[4:]]
    # Start to load a trained starspace model.
    # STARSPACE-2018-2
    # Model loaded.
    # Predictions use 386720 known labels.
    print(len(embeddings_raw), "lines loaded")

2220670 lines loaded

In [133]:
from itertools import zip_longest
def grouper(n, iterable, padvalue=None):
    # Thanks https://stackoverflow.com/a/312644
    "grouper(3, 'abcdefg', 'x') --> ('a','b','c'), ('d','e','f'), ('g','x','x')"
    return zip_longest(*[iter(iterable)]*n, fillvalue=padvalue)

In [134]:
embedding_pairs = list(grouper(2, embeddings_raw))

In [143]:
embedding_pairs[0]

('6501',
 '-0.199245 -0.0506518 0.00747807 0.0715903 0.056363 -0.0688545 -0.0966344 0.0375396 -0.0623841 0.156846 0.0796977 3.30169e-05 0.0803112 -0.0346252 -0.0889945 -0.00975803 0.155911 -0.0404718 0.0593154 0.101038 -0.0251951 0.0377808 -0.0527409 -0.0896889 -0.0393512 -0.0245343 0.17143 -0.112598 -0.158899 0.080887 0.16115 0.0500992 -0.020489 0.0102698 0.0739828 0.0882712 -0.0145368 -0.0675162 -0.034997 -0.0389806 0.0583331 -0.0580784 -0.0123978 -0.14178 0.0909726 0.0249295 0.0715458 -0.0672207 -0.0395715 0.103535 0.0444202 0.107718 -0.078376 0.0341841 -0.00703742 -0.0258491 0.0283285 -0.0510868 -0.1341 -0.0572724 0.169065 0.16329 -0.0542815 0.108796 0.121971 -0.0169707 0.206417 -0.034578 -0.125401 -0.0678717 0.0307413 -0.126458 -0.0831852 0.0798361 -0.0690379 -0.240386 0.101779 -0.054327 -0.10877 -0.00157392 0.215158 -0.0585009 -0.285273 -0.0959894 -0.120163 0.0324156 0.0200307 0.0756687 -0.103217 -0.0243171 0.241266 -0.0334473 0.0387139 -0.193606 -0.0962382 0.0283752 0.0437517 -0

In [144]:
embeddings = []
for embedding in embedding_pairs:
    embeddings.append((int(embedding[0]), [float(i) for i in embedding[1].split(" ")]))

In [146]:
load_dotenv()

True

In [ ]:
from qdrant_client import AsyncQdrantClient, models
qdrant = AsyncQdrantClient(
        url=environ['QDRANT_HOST'],
        port=int(environ['QDRANT_PORT']),
        grpc_port=int(environ['QDRANT_GRPC_PORT']),
        prefer_grpc=True,
        api_key=(environ['QDRANT_API_KEY']),
        
    )
COLLECTION_NAME = "wpa_recommendations"

/home/thel/code/projects/personal/Stylography/src/.venv/lib/python3.10/site-packages/qdrant_client/async_qdrant_remote.py:117: UserWarning: Api key is used with an insecure connection.
  warnings.warn("Api key is used with an insecure connection.")


In [150]:
if not await qdrant.collection_exists(COLLECTION_NAME):
    await qdrant.create_collection(
                collection_name=COLLECTION_NAME,
                vectors_config=models.VectorParams(
                    size=len(embeddings[0][1]),
                    distance=models.Distance.COSINE,
                ),
            )

E0000 00:00:1733063598.354600 4078513 ssl_transport_security.cc:1659] Handshake failed with fatal error SSL_ERROR_SSL: error:100000f7:SSL routines:OPENSSL_internal:WRONG_VERSION_NUMBER
E0000 00:00:1733063616.802554 4078514 ssl_transport_security.cc:1659] Handshake failed with fatal error SSL_ERROR_SSL: error:100000f7:SSL routines:OPENSSL_internal:WRONG_VERSION_NUMBER


In [ ]:
for idx, chunk in enumerate(grouper(10000, embeddings)):
    await qdrant.upsert(
            collection_name=COLLECTION_NAME,
            points=[
                models.PointStruct(id=id, vector=embedding)
                for id, embedding in chunk
            ],
)
    print("on chunk", idx)